In [0]:
from pyspark.sql import functions as F, types as T
import json

dbutils.widgets.text("dt", "")
dbutils.widgets.text("bucket", "")
dbutils.widgets.text("bronze_prefix", "")
dbutils.widgets.text("silver_prefix", "")
dbutils.widgets.dropdown("table", "chain_daily_state", ["chain_daily_state", "active_locks_daily"])






In [0]:
dt = dbutils.widgets.get("dt")
bucket = dbutils.widgets.get("bucket").strip()
bronze_prefix = dbutils.widgets.get("bronze_prefix").strip().rstrip("/")
silver_prefix = dbutils.widgets.get("silver_prefix").strip().rstrip("/")
table = dbutils.widgets.get("table")

if not bucket:
    raise ValueError("bucket is required")

bronze_dt = f"s3://{bucket}/{bronze_prefix}/dt={dt}"
manifest_path = f"{bronze_dt}/_MANIFEST.json"



In [0]:
%sql

DESCRIBE EXTERNAL LOCATION `db_s3_external_databricks-s3-optio-bronze-raw`;


GRANT READ FILES
ON EXTERNAL LOCATION `db_s3_external_databricks-s3-optio-bronze-raw`
TO `optio-devs`;

SHOW GRANTS ON EXTERNAL LOCATION `db_s3_external_databricks-s3-optio-bronze-raw`;

In [0]:
def read_manifest(path: str) -> dict:
    txt = dbutils.fs.head(path, 1000000)
    return json.loads(txt)

manifest = read_manifest(manifest_path)

display(manifest)
